In [1]:
import math, csv
import numpy as np
import scipy as sp
import pandas as pd
import pdb

from ml_utilities import *

from sklearn.preprocessing import Imputer

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
tr=pd.read_csv('train.csv')
te=pd.read_csv('test.csv')

target = tr['target']
tr = tr.drop(['ID','target'],axis=1)
IDs = te['ID'].values
te = te.drop(['ID'],axis=1)

# This seems to be a canonical list of data to avoid...
#tr = tr.drop(['v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)
#te = te.drop(['v8','v23','v25','v31','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128'],axis=1)

tr.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,1.050328,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [3]:
# Some basic parameters that we'll use a lot
n_tr = tr.shape[0]
n_te = te.shape[0]
n_keys = tr.shape[1]
print n_tr, n_te, n_keys

# Some basic settings that we'll use a lot
predictor_freq_min_ratio = 10
predictor_unique_min_ratio = 20
use_dummies = True
use_random_imputation = False

show_histogram = True

skew_z_thresh = 100.0 # Arbitrary choice, looks legit from histograms

log_trans = False
log_trans_marginals = True
log_trans_shift = 1.0e-6

tr_full, tr_null, tr_full_row, tr_null_row = split_by_nulls(tr)
te_full, te_null, te_full_row, te_null_row = split_by_nulls(te)

print "Training set with full rows size:", tr_full.shape
print "Training set with NaN rows size:", tr_null.shape
print "Test set with full rows size:", te_full.shape
print "Test set with NaN rows size:", te_null.shape

#print tr_full.head()
#print tr_null.head()

# We replace tr and te so the rest of the workbook works out...
tr = tr_full
te = te_full

114321 114393 131
Training set with full rows size: (64519, 131)
Training set with NaN rows size: (49802, 50)
Test set with full rows size: (64544, 131)
Test set with NaN rows size: (49849, 50)


# Data preprocessing
## First we look at distributions (before imputing phase)

In [ ]:
if show_histogram:
    pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
    pd.DataFrame.hist(tr.join(np.log(tr.select_dtypes(exclude=[object])[tr > 1.0e-5]), lsuffix="_n", rsuffix="_log"),
                      figsize=[30,140], layout=[40,6], bins=40)

removable_predictors = []
skewed_predictors = []

for (tr_col_name, tr_col), (te_col_name, te_col) in zip(tr.select_dtypes(exclude=[object]).iteritems(),\
                                                        te.select_dtypes(exclude=[object]).iteritems()):
    if te_col_name != tr_col_name:
        print "Uh oh, mismatched columns", tr_col_name, te_col_name
        
    # We add columns to a list of "removables" if there is too large a ratio between the two most common elements
    tr_val_counts = tr_col.value_counts().ravel()
    te_val_counts = te_col.value_counts().ravel()
    if len(tr_col.unique()) > 2:
        if tr_val_counts[0] / tr_val_counts[1] > predictor_freq_min_ratio and float(n_tr) / float(len(tr_col.unique())) > predictor_unique_min_ratio \
           and te_val_counts[0] / te_val_counts[1] > predictor_freq_min_ratio and float(n_te) / float(len(te_col.unique())) > predictor_unique_min_ratio:
            # The most frequent value is vastly more frequent than the next, in both the training and testing set
            removable_predictors.append(tr_col_name)
            print "Removable predictor? Three training set most common elements:\n", tr_col.value_counts()[tr_col.value_counts().keys()[:3]]

    # We add columns to a list of "skewed" data
    [z,p] = sp.stats.skewtest(tr_col.dropna().ravel())
    if z > skew_z_thresh:
        skewed_predictors.append(tr_col_name)
    
#
    
print removable_predictors
print skewed_predictors

# These "log-transformable" predictors are chosen simply from inspecting the histograms below...
log_transform = ['v97', 'v95', 'v94', 'v92', 'v8', 'v88', 'v86', 'v85', 'v84', 'v83', 'v76', 'v73',
                 'v60', 'v55', 'v50', 'v46', 'v39', 'v33', 'v32', 'v25', 'v23', 'v1', 'v19', 'v18', 
                 'v17', 'v15', 'v13', 'v131', 'v130', 'v128', 'v127','v126', 'v124', 'v123', 'v121', 
                 'v120', 'v119', 'v111', 'v109', 'v105', 'v102']
marginal_logs = ['v89', 'v63', 'v54', 'v27', 'v26', 'v118', 'v104']

if log_trans:
    # Then transform all those columns we just listed
    # Note that we could use:
    # from scipy.stats import boxcar
    # BoxCox_Transformed_Data = boxcox(OriginalData)
    # and also see: http://shahramabyari.com/2015/12/21/data-preparation-for-predictive-modeling-resolving-skewness/
    
    for col in log_transform:
        if col in tr.keys():
            tr[col] = np.log(tr[col]+log_trans_shift)
            te[col] = np.log(te[col]+log_trans_shift)
        
    if log_trans_marginals:
        for col in marginal_logs:
            if col in tr.keys():
                tr[col] = np.log(tr[col]+log_trans_shift)
                te[col] = np.log(te[col]+log_trans_shift)
                
                

for (tr_col_name, tr_col), (te_col_name, te_col) in zip(tr.iteritems(),te.iteritems()):
    if tr_col.dtype == 'object':
        tr_u = tr_col.unique()
        te_u = te_col.unique()
        if all([u not in te_u for u in tr_u]):
            print tr_col.unique()
            print te_col.unique()
        else:
            print tr_col_name, "has same unique values in both"

In [4]:
# Find all the columns with string data
tr = randomised_imputer(tr)
te = randomised_imputer(te)
tr_null = randomised_imputer(tr_null)
te_null = randomised_imputer(te_null)

#imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
#imp.fit(tr)
#imp.transform(tr)            

Filling for 30 NaNs in data v1
Filling for 0 NaNs in data v2
Filling for 2773 NaNs in object data v3
Adding 4 dummies for v3
Filling for 0 NaNs in data v4
Filling for 879 NaNs in data v5
Filling for 30 NaNs in data v6
Filling for 30 NaNs in data v7
Filling for 874 NaNs in data v8
Filling for 49 NaNs in data v9
Filling for 14 NaNs in data v10
Filling for 34 NaNs in data v11
Filling for 14 NaNs in data v12
Filling for 30 NaNs in data v13
Filling for 1 NaNs in data v14
Filling for 34 NaNs in data v15
Filling for 93 NaNs in data v16
Filling for 0 NaNs in data v17
Filling for 30 NaNs in data v18
Filling for 41 NaNs in data v19
Filling for 38 NaNs in data v20
Filling for 324 NaNs in data v21
Filling for 230 NaNs in object data v22
Numerising v22
Filling for 879 NaNs in data v23
Adding 5 dummies for v24
Filling for 874 NaNs in data v25
Filling for 30 NaNs in data v26
Filling for 30 NaNs in data v27
Filling for 30 NaNs in data v28
Filling for 30 NaNs in data v29
Filling for 23837 NaNs in objec

In [ ]:
import xgboost as xgb
xgtr = xgb.DMatrix(tr.values, target.ix[tr_full_row].values)
xgte = xgb.DMatrix(te.values)
xgtr_null = xgb.DMatrix(tr_null.values, target.ix[tr_null_row].values)
xgte_null = xgb.DMatrix(te_null.values)

params = {'objective': 'binary:logistic', 
          'subsample': 1, 
          'eta': 0.1,                    # eta : step size shrinkage on each boosting step
          #'gamma': 0.0,                 # minimum loss reduction requireed to make a further partition
          #'lambda': 1.0,                # L2 regularization term on weights
          #'alpha': 0.0,                 # L1 regularization term on weights
          'colsample_bytree': 0.9,       # subsample ratio of columns when constructing each tree
          'max_depth': 10,               # maximum depth of a single tree, default = 6
          'min_child_weight' : 5,        # minimum sum of weight needed in a child, default = 1
          'silent': 0 }

boost_round = 50
#clf = xgb.cv(params, xgtr, num_boost_round=num_round, metrics={'logloss'}, nfold=5,
#             seed=0, maximize=False)

clf = xgb.train(params, xgtr, num_boost_round=boost_round, verbose_eval=True, maximize=False)
clf_null = xgb.train(params, xgtr_full, num_boost_round=boost_round, verbose_eval=True, maximize=False)

# Make predictions
print('Predict...')
te_pred_full = clf.predict(xgte, ntree_limit=clf.best_iteration)
te_pred_null = clf.predict(xgte_null, ntree_limit=clf.best_iteration)
te_pred = pd.concat([te_pred_full, te_pred_null])
pdb.set_trace()
# Save results
predictions_file = open("simple_xgboost_result.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])

open_file_object.writerows(zip(IDs, te_pred))
predictions_file.close()
#
print('Done.')

In [ ]:
print tr.keys()
#print tr_keys_orig

#[tr_fac, tr_lab] = pd.factorize(tr['v22'])
#tr['v22'] = tr_fac
#tr['v56'].unique()

In [ ]:
# Testing out some techniques on dataframes
n = 10
choices = range(n/2)
df = pd.DataFrame({'a': np.random.normal(0,1,n), 'b' : np.random.choice(choices,n)})
print df
print df['b'].unique()
print np.bincount(df['b'])
print np.float32(np.bincount(df['b'])) /len(df['b'])
print np.float32(df['b'].value_counts().ravel()) / len(df['b'])

np.random.choice(df['b'].value_counts().keys().ravel(), size=3,
                             p = np.float32(df['b'].value_counts().ravel()) / df['b'].value_counts().ravel().sum() )

#unique_values = np.sort(tr.ix[:,i].unique())    
#if unique_values.dtype in [np.int64, np.float64]:
#    field_hist, field_bins = np.histogram(tr.ix[:,i], unique_values)